In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
pd.set_option('display.max_columns', None)

In [4]:
class SimpleNN(nn.Module):
    def __init__(self, firstLayerCount):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(firstLayerCount, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 2)  # Assuming you have 2 output classes
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x
class MyDataset(Dataset):
    def __init__(self, pd_dataFrame):
        self.data = pd_dataFrame
        self.X = self.data.iloc[:, :-1].values  # all columns except last
        self.y = self.data.iloc[:, -1].values   # last column as label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

In [7]:
# Initialize model and move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
df = pd.read_csv("Trim_train/split_part_1.csv")
# threshold = 0.95  # Keep only columns with ≥ 5% data
# df = df.dropna(axis=1, thresh=int(threshold * len(df)))
df = df.drop("ID", axis=1)
df = df.fillna(df.mean())

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = MyDataset(train_df)
test_dataset = MyDataset(test_df)
# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [12]:
# Assuming your SimpleNN model and device setup from earlier
model = SimpleNN(len(train_df.columns)-1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # You can adjust this

for epoch in range(num_epochs):
    model.train()  # set model to training mode
    running_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/10], Loss: nan
Epoch [2/10], Loss: nan
Epoch [3/10], Loss: nan
Epoch [4/10], Loss: nan
Epoch [5/10], Loss: nan
Epoch [6/10], Loss: nan
Epoch [7/10], Loss: nan
Epoch [8/10], Loss: nan
Epoch [9/10], Loss: nan
Epoch [10/10], Loss: nan


In [13]:
for X_batch, y_batch in train_loader:
    print("X_batch shape:", X_batch.shape)
    print("y_batch shape:", y_batch.shape)
    break

X_batch shape: torch.Size([64, 10212])
y_batch shape: torch.Size([64])


In [13]:
# check test score

model.eval()  # Set model to evaluation mode

y_true = []  # Store actual labels
y_pred = []  # Store predicted labels

with torch.no_grad():  # No gradient computation for inference
    for X_test, y_test in test_loader:
        X_test, y_test = X_test.to(device), y_test.to(device)

        # Forward pass
        outputs = model(X_test)

        # Get predicted class (index of max probability)
        _, predicted = torch.max(outputs, 1)

        # Store values
        y_true.extend(y_test.cpu().numpy())  # Move to CPU and convert to NumPy
        y_pred.extend(predicted.cpu().numpy())


precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.000000
Recall: 0.000000
F1 Score: 0.0000


C:\Users\Lu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
